In [ ]:
import sys
import numpy as np
import xarray as xr
import pandas as pd
import warnings
import geopandas
import matplotlib
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

sys.path.append("..")
# matplotlib.rc_file('../rc_file')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
warnings.filterwarnings("ignore", category=matplotlib.MatplotlibDeprecationWarning)

from geometry import ModelGeometry, glaciers
from PICO import PicoModel, table2
from compare_models import compare_PICO

# The PICO model
differences to Reese et al (2018):
- underlying datasets
- model resolution
- 

In [ ]:
for i, glacier in enumerate(glaciers):
    if glacier in ['Ross', 'FilchnerRonne']:  continue
    PicoModel(glacier).compute_pico()

In [ ]:
compare_PICO()

### maps of Amundsen Sea and East Antarctica

In [ ]:
proj = ccrs.SouthPolarStereo(true_scale_latitude=-71)
def fn_poly(glacier):  return f'../../data/mask_polygons/{glacier}_polygon.geojson'
x5, y5, _, _ = geopandas.read_file(fn_poly('MoscowUniversity'), crs='espg:3031').total_bounds
_, _, x6, y6 = geopandas.read_file(fn_poly('Totten'), crs='espg:3031').total_bounds
x3, _, _, y4 = geopandas.read_file(fn_poly('PineIsland'), crs='espg:3031').total_bounds
_, y3, x4, _ = geopandas.read_file(fn_poly('Dotson'), crs='espg:3031').total_bounds

import matplotlib.ticker as mticker

f = plt.figure(figsize=(8,12))  # Amundsen Sea
for i in range(2):
    (x1,x2,y1,y2) = [(x3,x4,y3-1e4,y4+2e4),(x5-1e4,x6,y5,y6+1e4)][i]
    shelves = [['PineIsland','Thwaites','Dotson'], ['Totten','MoscowUniversity']][i]
    for s, shelf in enumerate(shelves):
        (x,y) = [[(.65,.88),(.05,.55),(.05,.2)],[(.3,.8),(.4,.1)]][i][s]
        name = [['Pine\nIsland','Thwaites','Dotson/\nCrosson'], ['Totten','Moscow\nUniversity']][i][s]
        dsg = xr.open_dataset(ModelGeometry(shelf).fn_PICO)
        dsP = xr.open_dataset(PicoModel(shelf).fn_PICO_output)
        lon, lat = dsg.lon, dsg.lat
        for j in range(3):
            q = [dsg.draft, dsg.box.where(dsg.mask), dsP.melt.where(dsg.mask)][j]
            cmap = ['viridis', 'Spectral','inferno_r'][j]
            (vmin,vmax) = [(-2000,0),(1,2),(0,25)][j]
            ax = f.add_axes([j/3,.545-.54*i,.33,.45], projection=proj)
            ax.set_frame_on(False)
            ax.set_extent([x1,x2,y1,y2], crs=proj)
            ax.coastlines()
            gl = ax.gridlines()
            
            gl.xlocator = mticker.FixedLocator(np.arange(-180,179,5))
            gl.ylocator = mticker.FixedLocator(np.arange(-89,89))
            
            im = ax.pcolormesh(lon, lat, q, transform=ccrs.PlateCarree(),
                          cmap=cmap, vmin=vmin, vmax=vmax)
            if i==0:  # colorbars
                cax = f.add_axes([j/3+.02,.5,.29,.02])
                label = ['draft [m]', 'box nr.', 'melt rate [m/yr]'][j]
                plt.colorbar(im, cax=cax, orientation='horizontal', label=label)
            if j==0:  ax.text(x, y, name, weight='bold', transform=ax.transAxes)
            if j==2:  ax.text(x, y, f'{dsP.mk[0].values:.2f} m/yr', transform=ax.transAxes)

## testcase: rectangular ice shelf

In [ ]:
nx, ny = 10, 3
mask = np.ones((ny,nx))
draft, _ = np.meshgrid(np.linspace(-1000,-500,nx), np.ones((ny)))
grl = np.zeros((ny,nx))
isf = np.zeros((ny,nx))
grl[:,0] = 1
isf[:,-1] = 1
kwargs = {'dims':['y','x'], 'coords':{'x':np.linspace(0,1e5,nx), 'y':np.linspace(0,1e5,ny)}}
da0 = xr.DataArray(data=draft, name='draft', **kwargs)
da1 = xr.DataArray(data=isf  , name='isf'  , **kwargs)
da2 = xr.DataArray(data=grl  , name='grl'  , **kwargs)
da3 = xr.DataArray(data=mask , name='mask' , **kwargs)
da = xr.merge([da0, da1, da2, da3])

In [ ]:
ds = PicoModel(name='test', Ta=Ta, Sa=Sa, n=3, ds=da).compute()

In [ ]:
f, ax = plt.subplots(4, 3, figsize=(15,15))
for i, key in enumerate(list(ds.keys())[:-1]):
    if i<9:  kwargs = {'cbar_kwargs':{'orientation':'horizontal'}}
    else:    kwargs = {}
    ds[key].plot(ax=ax[int(i/3), i%3], **kwargs)